<a href="https://colab.research.google.com/github/vincimech010233/QuantumComputingJourney-/blob/main/Badge-Pennylane/Optimus_Trine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 24.1 MB/s eta 0:00:00


In [ ]:
import json
import pennylane as qml
import pennylane.numpy as np

def circuit_left():
    """
    This function corresponds to the circuit on the left-hand side of the diagram in the
    description. Simply place the necessary operations, you do not have to return anything.
    """
    # Using 3-qubit bit flip error correction https://quantumcomputinguk.org/tutorials/quantum-error-correction-bit-flip-code-in-qiskit
    qml.CNOT([0, 1])

    # CNOT 0, 2
    qml.SWAP([0, 1])
    qml.CNOT([1, 2])
    qml.SWAP([0, 1])

def circuit_right():
    """
    This function corresponds to the circuit on the right-hand side of the diagram in the
    description. Simply place the necessary operations, you do not have to return anything.
    """
    qml.CNOT([0, 1])

    # CNOT 0, 2
    qml.SWAP([0, 1])
    qml.CNOT([1, 2])
    qml.SWAP([0, 1])

    # TOFFOLLI single gate decomposition (target wire 0, controlled on wires 1 and 2)
    qml.Hadamard(0)
    qml.CNOT([1, 0])
    qml.adjoint(qml.T(0))

    # CNOT 2, 0
    qml.SWAP([2, 1])
    qml.CNOT([1, 0])
    qml.SWAP([2, 1])

    qml.T(0)
    qml.CNOT([1, 0])
    qml.adjoint(qml.T(0))

    # CNOT 2, 0
    qml.SWAP([2, 1])
    qml.CNOT([1, 0])
    qml.SWAP([2, 1])

    qml.adjoint(qml.T(1))
    qml.T(0)
    qml.CNOT([2, 1])
    qml.Hadamard(0)
    qml.adjoint(qml.T(1))
    qml.CNOT([2, 1])
    qml.T(2)
    qml.S(1)

    # Swap state of first qubit with third qubit
    qml.SWAP([0, 1])
    qml.SWAP([1, 2])

def U():
    """This operator generates a PauliX gate on a random qubit"""
    qml.PauliX(wires=np.random.randint(3))


dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def circuit(alpha, beta, gamma):
    """Total circuit joining each block.

    Args:
        alpha (float): The first parameter of a U3 gate.
        beta (float):The second parameter of a U3 gate.
        gamma (float): The third parameter of a U3 gate.

    Returns:
        (float): The expectation value of an observable.
    """
    qml.U3(alpha, beta, gamma, wires=0)
    circuit_left()
    U()
    circuit_right()

    # Here we are returning the expected value with respect to any observable,
    # the choice of observable is not important in this exercise.

    return qml.expval(0.5 * qml.PauliZ(2) - qml.PauliY(2))


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    angles = json.loads(test_case_input)
    output = circuit(*angles)
    return str(output)

def check(solution_output: str, expected_output: str) -> None:

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, atol=2e-1
    ), "The expected output is not quite right."

    ops = circuit.tape.operations

    for op in ops:
        assert not (0 in op.wires and 2 in op.wires), "Invalid connection between qubits."

    assert circuit.tape.observables[0].wires == qml.wires.Wires(2), "Measurement on wrong qubit."


test_cases = [['[2.0,1.0,3.0]', '-0.97322'], ['[-0.5,1.2,-1.2]', '0.88563'], ['[0.22,3.0,2.1]', '0.457152'], ['[2.22,3.1,-3.3]', '-0.335397'], ['[-0.2,-0.1,3.4]', '0.470199'], ['[-1.2,-1.1,0.4]', '-0.6494612']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[2.0,1.0,3.0]'...
Correct!
Running test case 1 with input '[-0.5,1.2,-1.2]'...
Correct!
Running test case 2 with input '[0.22,3.0,2.1]'...
Correct!
Running test case 3 with input '[2.22,3.1,-3.3]'...
Correct!
Running test case 4 with input '[-0.2,-0.1,3.4]'...
Correct!
Running test case 5 with input '[-1.2,-1.1,0.4]'...
Correct!


In [ ]:
import json
import pennylane as qml
import pennylane.numpy as np

dev = qml.device("default.qubit", wires=["e1", "e2", "e3", "e4", "result"], shots=1)

wires = ["e1", "e2", "e3", "e4", "result"]

@qml.qnode(dev)
def circuit(project_execution):
    """This is the circuit we will use to detect which is the lazy worker. Remember
    that we will only execute one shot.

    Args:
        project_execution (qml.ops):
            The gate in charge of marking in the last qubit if the project has been finished
            as indicated in the statement.

    Returns:
        (numpy.tensor): Measurement output in the 5 qubits after a shot.
    """
    # Prepare the initial state
    state = [0] * (2**5)
    state[14] = 1/2
    state[22] = 1/2
    state[26] = 1/2
    state[28] = 1/2
    qml.QubitStateVector(state, wires=["e1", "e2", "e3", "e4", "result"])

    # Apply operations
    qml.PauliX(wires="result")
    qml.Hadamard(wires="result")

    # Call the project_execution operator
    project_execution(wires=wires)

    qml.Hadamard(wires="result")
    qml.PauliX(wires="result")

    # Apply unitary operation
    U = np.identity(2**5)
    U[14, 14] = 0
    U[14, 0] = 1
    U[0, 14] = 1
    U[0, 0] = 0

    U[22, 22] = 0
    U[22, 8] = 1
    U[8, 22] = 1
    U[8, 8] = 0

    U[26, 26] = 0
    U[26, 16] = 1
    U[16, 26] = 1
    U[16, 16] = 0

    U[28, 28] = 0
    U[28, 24] = 1
    U[24, 28] = 1
    U[24, 24] = 0

    qml.QubitUnitary(U, wires=["e1", "e2", "e3", "e4", "result"])

    U1 = np.array([[-1, 1, 1, 1], [1, -1, 1, 1], [1, 1, -1, 1], [1, 1, 1, -1]]) / 2
    qml.QubitUnitary(U1, wires=["e1", "e2"])

    return qml.sample(wires=dev.wires)

def process_output(measurement):
    """This function will take the circuit measurement and process it to determine who is the lazy worker.

    Args:
        measurement (numpy.tensor): Measurement output in the 5 qubits after a shot.

    Returns:
        (str): This function must return "e1", "e2", "e3" or "e4" - the lazy worker.
    """
    # Convert the measurement to a list
    measurement = list(measurement)
    if measurement == [0, 0, 0, 0, 0]:
        return "e1"
    if measurement == [0, 1, 0, 0, 0]:
        return "e2"
    if measurement == [1, 0, 0, 0, 0]:
        return "e3"
    if measurement == [1, 1, 0, 0, 0]:
        return "e4"

# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    return None

def check(solution_output: str, expected_output: str) -> None:
    samples = 5000

    solutions = []
    output = []

    for s in range(samples):
        lazy = np.random.randint(0, 4)
        no_lazy = list(range(4))
        no_lazy.pop(lazy)

        def project_execution(wires):
            class op(qml.operation.Operator):
                num_wires = 5

                def compute_decomposition(self, wires):
                    raise ValueError("You can't decompose this gate")

                def matrix(self):
                    m = np.zeros([32, 32])
                    for i in range(32):
                        b = [int(j) for j in bin(64 + i)[-5:]]
                        if sum(np.array(b)[no_lazy]) == 3:
                            if b[-1] == 0:
                                m[i, i + 1] = 1
                            else:
                                m[i, i - 1] = 1
                        else:
                            m[i, i] = 1
                    return m

            op(wires=wires)
            return None

        out = circuit(project_execution)
        solutions.append(lazy + 1)
        output.append(int(process_output(out)[-1]))

    assert np.allclose(
        output, solutions, rtol=1e-4
    ), "Your circuit does not give the correct output."

    ops = [op.name for op in circuit.tape.operations]
    assert ops.count("op") == 1, "You have used the oracle more than one time."

# These are the public test cases
test_cases = [
    ('No input', 'No output')
]
# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")


Running test case 0 with input 'No input'...
Correct!


In [ ]:
@qml.qnode(dev)
def circuit(project_execution):
    """This is the circuit we will use to detect which is the lazy worker. Remember
    that we will only execute one shot.

    Args:
        project_execution (qml.ops):
            The gate in charge of marking in the last qubit if the project has been finished
            as indicated in the statement.

    Returns:
        (numpy.tensor): Measurement output in the 5 qubits after a shot.
    """

    # Put your code here #

    ## This is Grover search over the reduced 4-dimensional subspace
    # init |0111>|0> + |1011>|0> + |1101>|0> + |1110>|0>
    n = len(wires) - 1
    for i in range(n): # prep i-th qubit
        if i == 0: # non-controlled RX
            qml.RX(np.arcsin(1/np.sqrt(n-i)) * 2, wires=wires[i])
        else: # multi-controlled RX
            qml.ctrl(qml.RX, control=wires[:i])(
                np.arcsin(1/np.sqrt(n-i)) * 2, wires[i]
            )
        qml.PauliX(wires=wires[i]) # rotate i-th qubit to 1

    # phase flip the solution
    # e.g. -|0111>|-> + |1011>|-> + |1101>|-> + |1110>|->
    qml.PauliX(wires="result")
    qml.Hadamard(wires="result")
    project_execution(wires=wires)

    # reverse the initialisation
    for i in range(n)[::-1]: # prep i-th qubit
        qml.PauliX(wires=wires[i])
        if i == 0: # non-controlled RX
            qml.RX(-np.arcsin(1/np.sqrt(n-i)) * 2, wires=wires[i])
        else: # multi-controlled RX
            qml.ctrl(qml.RX, control=wires[:i])(
                -np.arcsin(1/np.sqrt(n-i)) * 2, wires[i]
            )

    # phase flip all the initial state |0000>
    for i in wires[:-1]:
        qml.PauliX(wires=i)
    qml.ctrl(qml.PauliZ, control=wires[:-2])(wires=wires[-2])
    for i in wires[:-1]:
        qml.PauliX(wires=i)

    # redo the initialisation
    for i in range(n): # prep i-th qubit
        if i == 0: # non-controlled RX
            qml.RX(np.arcsin(1/np.sqrt(n-i)) * 2, wires=wires[i])
        else: # multi-controlled RX
            qml.ctrl(qml.RX, control=wires[:i])(
                np.arcsin(1/np.sqrt(n-i)) * 2, wires[i]
            )
        qml.PauliX(wires=wires[i])

    return qml.sample(wires=dev.wires[:-1])

def process_output(measurement):
    """This function will take the circuit measurement and process it to determine who is the lazy worker.

    Args:
        measurement (numpy.tensor): Measurement output in the 5 qubits after a shot.

    Returns:
        (str): This function must return "e1", "e2", "e3", or "e4" - the lazy worker.
    """
    if np.all(measurement == [0,1,1,1]):
        return "e1"
    elif np.all(measurement == [1,0,1,1]):
        return "e2"
    elif np.all(measurement == [1,1,0,1]):
        return "e3"
    else:
        return "e4"
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    return None

def check(solution_output: str, expected_output: str) -> None:
    samples = 5000

    solutions = []
    output = []

    for s in range(samples):
        lazy = np.random.randint(0, 4)
        no_lazy = list(range(4))
        no_lazy.pop(lazy)

        def project_execution(wires):
            class op(qml.operation.Operator):
                num_wires = 5

                def compute_decomposition(self, wires):
                    raise ValueError("You cant descompose this gate")

                def matrix(self):
                    m = np.zeros([32, 32])
                    for i in range(32):
                        b = [int(j) for j in bin(64 + i)[-5:]]
                        if sum(np.array(b)[no_lazy]) == 3:
                            if b[-1] == 0:
                                m[i, i + 1] = 1
                            else:
                                m[i, i - 1] = 1
                        else:
                            m[i, i] = 1
                    return m

            op(wires=wires)
            return None

        out = circuit(project_execution)
        solutions.append(lazy + 1)
        output.append(int(process_output(out)[-1]))

    assert np.allclose(
        output, solutions, rtol=1e-4
    ), "Your circuit does not give the correct output."

    ops = [op.name for op in circuit.tape.operations]
    assert ops.count("op") == 1, "You have used the oracle more than one time."
test_cases = [['No input', 'No output']]
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input 'No input'...
Correct!


In [ ]:
import json
import pennylane as qml
import pennylane.numpy as np

num_wires = 4
dev = qml.device("default.mixed", wires=num_wires)

@qml.qnode(dev)
def heisenberg_trotter(couplings, p, time, depth):
    """This QNode returns the final state of the spin chain after evolution for a time t,
    under the Trotter approximation of the exponential of the Heisenberg Hamiltonian.

    Args:
        couplings (list(float)):
            An array of length 4 that contains the coupling constants and the magnetic field
            strength, in the order [J_x, J_y, J_z, h].
        p (float): The depolarization probability after each CNOT gate.
        depth (int): The Trotterization depth.
        time (float): Time during which the state evolves

    Returns:
        (numpy.tensor): The evolved quantum state.
    """


    # Put your code here #
    J_x = -couplings[0]
    J_y = -couplings[1]
    J_z = -couplings[2]
    h = -couplings[3]
    N = 4

    qml.RY(0, wires=0)

    for i in range(depth):

        for j in range(N):
        #qml.PauliRot(2*J_x*time/(depth), 'XX',  [(j+1) % N,j])
            # Hadamard(wires=[(j+1) % N])
            qml.RZ(np.pi/2, wires=[(j+1) % N])
            qml.RX(np.pi/2, wires=[(j+1) % N])
            qml.RZ(np.pi/2, wires=[(j+1) % N])
            # Hadamard(wires=[j])
            qml.RZ(np.pi/2, wires=[j])
            qml.RX(np.pi/2, wires=[j])
            qml.RZ(np.pi/2, wires=[j])

            qml.CNOT(wires=[j, (j+1) % N])
            qml.DepolarizingChannel(p, (j+1) % N)

            qml.RZ(2*J_x*time/(depth),wires=[(j+1) % N])

            qml.CNOT(wires=[j, (j+1) % N])
            qml.DepolarizingChannel(p, (j+1) % N)

            # Hadamard(wires=[(j+1) % N])
            qml.RZ(np.pi/2, wires=[(j+1) % N])
            qml.RX(np.pi/2, wires=[(j+1) % N])
            qml.RZ(np.pi/2, wires=[(j+1) % N])
            # Hadamard(wires=[j])]
            qml.RZ(np.pi/2, wires=[j])
            qml.RX(np.pi/2, wires=[j])
            qml.RZ(np.pi/2, wires=[j])


        for j in range(N):
        #qml.PauliRot(2*J_y*time/(depth), 'YY',  [(j+1) % N, j])
           qml.RX(np.pi/2, wires=[(j+1) % N])
           qml.RX(np.pi/2, wires=[j])

           qml.CNOT(wires=[j, (j+1) % N])
           qml.DepolarizingChannel(p, (j+1) % N)

           qml.RZ(2*J_y*time/(depth), wires=[(j+1) % N])

           qml.CNOT(wires=[j, (j+1) % N])
           qml.DepolarizingChannel(p, (j+1) % N)

           qml.RX(-np.pi/2, wires=[(j+1) % N])
           qml.RX(-np.pi/2, wires=[j])

        for j in range(N):
        #qml.PauliRot(2*J_z*time/(depth), 'ZZ',  [(j+1) % N, j])
            qml.CNOT(wires=[j, (j+1) % N])
            qml.DepolarizingChannel(p, (j+1) % N)

            qml.RZ(2*J_z*time/(depth), wires=[(j+1) % N])

            qml.CNOT(wires=[j, (j+1) % N])
            qml.DepolarizingChannel(p, (j+1) % N)

        for j in range(N):
        #qml.PauliRot(2*h*time/(depth), 'X',  [j])
            # Hadamard(wires=[j])
            qml.RZ(np.pi/2, wires=[j])
            qml.RX(np.pi/2, wires=[j])
            qml.RZ(np.pi/2, wires=[j])

            qml.RZ(2*h*time/(depth), wires=[j])

            # Hadamard(wires=[j])
            qml.RZ(np.pi/2, wires=[j])
            qml.RX(np.pi/2, wires=[j])
            qml.RZ(np.pi/2, wires=[j])


    return qml.state()

def calculate_fidelity(couplings, p, time, depth):
    """This function returns the fidelity between the final states of the noisy and
    noiseless Trotterizations of the Heisenberg models, using only CNOT and rotation gates

    Args:
        couplings (list(float)):
            A list with the J_x, J_y, J_z and h parameters in the Heisenberg Hamiltonian, as
            defined in the problem statement.
        p (float): The depolarization probability of the depolarization gate that acts on the
                   target qubit of each CNOT gate.
        time (float): The period of time evolution simulated by the Trotterization.
        depth (int): The Trotterization depth.

    Returns:
        (float): Fidelity between final states of the noisy and noiseless Trotterizations
    """
    return qml.math.fidelity(heisenberg_trotter(couplings,0,time, depth),heisenberg_trotter(couplings,p,time,depth))


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:

    ins = json.loads(test_case_input)
    output =calculate_fidelity(*ins)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    """
    Compare solution with expected.

    Args:
            solution_output: The output from an evaluated solution. Will be
            the same type as returned.
            expected_output: The correct result for the test case.

    Raises:
            ``AssertionError`` if the solution output is incorrect in any way.

    """
    def create_hamiltonian(params):

        couplings = [-params[-1]]
        ops = [qml.PauliX(3)]

        for i in range(3):

            couplings = [-params[-1]] + couplings
            ops = [qml.PauliX(i)] + ops

        for i in range(4):

            couplings = [-params[-2]] + couplings
            ops = [qml.PauliZ(i)@qml.PauliZ((i+1)%4)] + ops

        for i in range(4):

            couplings = [-params[-3]] + couplings
            ops = [qml.PauliY(i)@qml.PauliY((i+1)%4)] + ops

        for i in range(4):

            couplings = [-params[0]] + couplings
            ops = [qml.PauliX(i)@qml.PauliX((i+1)%4)] + ops

        return qml.Hamiltonian(couplings,ops)

    @qml.qnode(dev)
    def evolve(params, time, depth):

        qml.ApproxTimeEvolution(create_hamiltonian(params), time, depth)

        return qml.state()

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    tape = heisenberg_trotter.qtape
    names = [op.name for op in tape.operations]

    random_params = np.random.uniform(low = 0.8, high = 3.0, size = (4,) )

    assert qml.math.fidelity(heisenberg_trotter(random_params,0,1,2),evolve(random_params,1,2)) >= 1, "Your circuit does not Trotterize the Heisenberg Model"

    assert names.count('ApproxTimeEvolution') == 0, "Your circuit must not use the built-in PennyLane Trotterization"

    assert set(names) == {'DepolarizingChannel', 'RX', 'RY', 'RZ', 'CNOT'}, "Your circuit must only use RX, RY, RZ, CNOT, and depolarizing gates (don't use qml.Rot or Paulis)"

    assert solution_output >= expected_output-0.005, "Your fidelity is not high enough. You may be using more CNOT gates than needed"


test_cases = [['[[1,2,1,0.3],0.05,2.5,1]', '0.33723981123369573'], ['[[1,3,2,0.3],0.05,2.5,2]', '0.15411351752086694']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[1,2,1,0.3],0.05,2.5,1]'...
Correct!
Running test case 1 with input '[[1,3,2,0.3],0.05,2.5,2]'...
Correct!


In [ ]:
import json
import pennylane as qml
import pennylane.numpy as np

dev = qml.device("default.qubit", wires=2, shots=1)

@qml.qnode(dev)
def circuit(U):
    """This will be the circuit you will use to determine which of the two angles we have.
    Remember that only a single shot will be executed.

    Args:
        U (qml.ops): It is the gate to discriminate between  RY(2pi/3) or RY(4pi/3).

    Returns:
        (numpy.tensor): Vector of two elements representing the output measured in each of the qubits.
    """
    # Prepare the control qubit in superposition
    qml.Hadamard(wires=0)

    # Apply controlled-U operation
    qml.ctrl(U, control=0)(wires=1)

    # Measure the qubits
    return qml.sample(wires=[0, 1])

def process_output(measurement):
    """This function processes the output of the circuit to discriminate between gates.

    Args:
        measurement (numpy.array): Output of the previous circuit a vector of dimension 2.

    Returns:
        (int): return 2 or 4 depending on the associated RY gate.
    """
    # If the measurement outcome is [1, 0] or [1, 1], it indicates RY(2pi/3)
    # If the measurement outcome is [0, 0] or [0, 1], it indicates RY(4pi/3)
    if measurement[0] == 1:
        return 2
    else:
        return 4

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    return None

def check(solution_output: str, expected_output: str) -> None:
    numbers = 2 * np.random.randint(1, 3, 5000)

    def U2(wires):
        class op(qml.operation.Operator):
            num_wires = 1

            def compute_decomposition(self, wires):
                raise ValueError("You cannot decompose this gate")

            def matrix(self):
                return qml.matrix(qml.RY(2 * np.pi / 3, wires=3))

        op(wires=wires)
        return None

    def U4(wires):
        class op(qml.operation.Operator):
            num_wires = 1

            def compute_decomposition(self, wires):
                raise ValueError("You cannot decompose this gate")

            def matrix(self):
                return qml.matrix(qml.RY(4 * np.pi / 3, wires=3))

        op(wires=wires)
        return None

    output = []
    for i in numbers:
        if i == 2:
            U = U2
        else:
            U = U4
        out = circuit(U)
        output.append(process_output(out))

    assert np.allclose(
        output, numbers, rtol=1e-4
    ), "Your circuit does not give the correct output."

# These are the public test cases
test_cases = [
    ('No input', 'No output')
]
# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        try:
            check(output, expected_output)
            print("Correct!")
        except AssertionError as e:
            print(f"Wrong Answer. {e}. Have: '{output}'. Want: '{expected_output}'.")


Running test case 0 with input 'No input'...
Wrong Answer. Your circuit does not give the correct output.. Have: 'None'. Want: 'No output'.


In [ ]:
import json
import pennylane as qml
import pennylane.numpy as np

dev = qml.device("default.qubit", wires=2, shots=1)

@qml.qnode(dev)
def circuit(U):
    """This will be the circuit you will use to determine which of the two angles we have.
    Remember that only a single shot will be executed.

    Args:
        U (qml.ops): It is the gate to discriminate between  RY(2pi/3) or RY(4pi/3).

    Returns:
        (numpy.tensor): Vector of two elements representing the output measured in each of the qubits.
    """
    # to use U,  call 'U(wires = <wire where you want to apply the gate>)'
    # to use Control-U, call 'qml.ctrl(U, control = <control wire>)(wires = <wire where you want to apply the gate>)'

    # Put your code here #

    qml.Hadamard(wires=0) # |00> + |10>
    for _ in range(3): # |00>+|10> for 4pi/3 or |00>-|10> for 2pi/3
        qml.ctrl(U, control=0)(wires=1)
    qml.Hadamard(wires=0) # |00> or |10>

    return qml.sample(wires=range(2))

def process_output(measurement):
    """This function processes the output of the circuit to discriminate between gates.

    Args:
        measurement (numpy.array): Output of the previous circuit a vector of dimension 2.

    Returns:
        (int): return 2 or 4 depending on the associated RY gate.
    """

    # Put your code here #

    if measurement[0] == 0:
        return 4
    else:
        return 2

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    return None

def check(solution_output: str, expected_output: str) -> None:
    numbers = 2 * np.random.randint(1, 3, 5000)

    def U2(wires):
        class op(qml.operation.Operator):
            num_wires = 1

            def compute_decomposition(self, wires):
                raise ValueError("You cannot decompose this gate")

            def matrix(self):
                return qml.matrix(qml.RY(2 * np.pi / 3, wires=3))

        op(wires=wires)
        return None

    def U4(wires):
        class op(qml.operation.Operator):
            num_wires = 1

            def compute_decomposition(self, wires):
                raise ValueError("You cannot decompose this gate")

            def matrix(self):
                return qml.matrix(qml.RY(4 * np.pi / 3, wires=3))

        op(wires=wires)
        return None

    output = []
    for i in numbers:
        if i == 2:
            U = U2
        else:
            U = U4
        out = circuit(U)
        output.append(process_output(out))

    assert np.allclose(
        output, numbers, rtol=1e-4
    ), "Your circuit does not give the correct output."

test_cases = [['No input', 'No output']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        try:
            check(output, expected_output)
            print("Correct!")
        except AssertionError as e:
            print(f"Wrong Answer. {e}. Have: '{output}'. Want: '{expected_output}'.")


Running test case 0 with input 'No input'...
Correct!


In [ ]:
import json
import pennylane as qml
from pennylane import numpy as np
# Uneditable section #

def zenda_operator():
    """
    Quantum function corresponding to the operator to be applied by
    Zenda in her qubits.This function does not return anything,
    you must simply write the necessary gates.
    """

    # End of uneditable section #

    # Put your code here #

    qml.CNOT(["z0","z1"])


# Uneditable section #
def reece_operator():
    """
    Quantum function corresponding to the operator to be applied by
    Reece in his qubits.This function does not return anything,
    you must simply write the necessary gates.
    """
    # End of uneditable section #

    # Put your code here #

    qml.CZ(["r0","r1"])


# Uneditable section #
def magic_operator():
    """
    Quantum function corresponding to the operator to be applied on the "z1"
    and "r1" qubits. This function does not return anything, you must
    simply write the necessary gates.

    """

    # End of uneditable section #

    # Put your code here #



    qml.CNOT(wires=["z1","r1"])
    qml.Hadamard(wires=["z1"])


# Uneditable section #
def bell_generator():
    """
    Quantum function preparing bell state shared by Reece and Zenda.
    """

    qml.Hadamard(wires=["z1"])
    qml.CNOT(wires=["z1", "r1"])


dev = qml.device("default.qubit", wires=["z0", "z1", "r1", "r0"])

@qml.qnode(dev)
def circuit(j, k):
    bell_generator()

    # j encoding and Zenda operation
    qml.BasisEmbedding([j], wires="z0")
    zenda_operator()

    # k encoding and Reece operation
    qml.BasisEmbedding([k], wires="r0")
    reece_operator()

    magic_operator()

    return qml.probs(wires=dev.wires)
def run(test_case_input: str) -> str:
    return None


def check(solution_output: str, expected_output: str) -> None:

    try:
        dev1 = qml.device("default.qubit", wires = ["z0", "z1"])
        @qml.qnode(dev1)
        def circuit1():
            zenda_operator()
            return qml.probs(dev1.wires)
        circuit1()
    except:
        assert False, "zenda_operator can only act on z0 and z1 wires"

    try:
        dev2 = qml.device("default.qubit", wires = ["r0", "r1"])
        @qml.qnode(dev2)
        def circuit2():
            reece_operator()
            return qml.probs(dev2.wires)
        circuit2()
    except:
        assert False, "reece_operator can only act on r0 and r1 wires"
    try:
        dev3 = qml.device("default.qubit", wires = ["z1", "r1"])
        @qml.qnode(dev3)
        def circuit3():
            magic_operator()
            return qml.probs(dev3.wires)
        circuit3()
    except:
        assert False, "magic_operator can only act on r1 and z1 wires"


    for j in range(2):
        for k in range(2):
            assert np.isclose(circuit(j, k)[10 * j + 5 * k], 1), "The output is not correct"
test_cases = [['No input', 'No output'], ['No input', 'No output']]
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input 'No input'...
Correct!
Running test case 1 with input 'No input'...
Correct!
